In [1]:
import requests 
import json 
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)

In [3]:
# Insert paths here
#data_path = "C:/Users/Joel/Downloads/Kickstarter_2021-04-15T03_20_08_451Z/Kickstarter.csv"
#driver_path = "C:/Users/Joel/Downloads/chromedriver_win32/chromedriver.exe"
data_path = "data/all_processed_df.csv"
driver_path = "/Users/benji/Downloads/chromedriver"
driver = webdriver.Chrome(driver_path)
#df = pd.read_csv("data/2021_04/Kickstarter.csv")
df = pd.read_parquet("data/all_processed_df.parquet.gzip")

In [4]:
df.columns

Index(['state', 'usd_goal', 'available_time', 'blurb_len', 'launched_at',
       'deadline', 'blurb', 'name', 'currency', 'country', 'is_starred',
       'is_starrable', 'spotlight', 'staff_pick', 'photo', 'urls', 'cat_id',
       'cat_position', 'cat_parent_id', 'cat_color', 'loc_id', 'loc_type',
       'loc_state'],
      dtype='object')

In [5]:
def proj_url(df, index):
    tmp = df.iloc[index]
    # is stored as a json type 
    url = json.loads(tmp['urls'])['web']['project']
    return url 

In [6]:
def load_text_url(url):
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, features="html.parser")
    # kill all script and style elements
    return soup 
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    # get text
    text = soup.get_text()
    return soup.find_all(text=True)

In [15]:
def extract_proj_descrip(url):
    # i think descriptions are pulled up w/ some java script bs so we need to load in driver
    driver.get(url)
    html = driver.page_source
    # load html 
    soup = BeautifulSoup(html, "lxml")
    proj = soup.find("section", {"class": "project-content"})
    if proj is not None: projtxt = proj.find_all(text=True)
    else: projtxt = None
    return proj, projtxt

In [8]:
def shorten_list(wrds):
    # words signal end of project description 
    beg = 'Story'
    # Set endings
    end1 = 'Check out the FAQ'
    end2 = 'Report this project to Kickstarter'
    end3 = 'Select this reward'
    end4 = 'Learn about accountability on Kickstarter'
    endings = [end1, end2, end3, end4]
    
    beg_ind = 0
    end_ind = len(wrds)
    
    for i in range(len(wrds)):
        if beg_ind == 0 and beg in wrds[i]:
            beg_ind = i
        elif beg_ind != 0 and any(x in wrds[i] for x in endings):
            end_ind = i
            return wrds[beg_ind:end_ind]
    
    #print("no end")
    return wrds[beg_ind:end_ind]

In [9]:
def clean_list(wrds):
    desc = []
    problem_chars = ['\n', '\r', '\xa0']
    for w in wrds:
        if any(x in w for x in problem_chars):
            for pc in problem_chars:
                w = w.replace(pc, '')
        if w != "":
            desc.append(w)
    return desc
        

In [10]:
def compress_list(wrds):
    desc = ''
    for w in wrds:
        desc = desc + w
    return desc

In [17]:
def load_proj_descrip(i):
    p, ptxt = extract_proj_descrip(proj_url(df, i))
    if p is not None: 
        short_ptxt = shorten_list(ptxt)
        clean_ptxt = clean_list(short_ptxt)
    else: clean_ptxt = []
    #proj_desc = compress_list(clean_ptxt)
    return clean_ptxt

In [18]:
tmp = df.iloc[0:500]

In [13]:
import time

In [19]:
start = time.time()
res = tmp.apply(lambda row: load_proj_descrip(row.name), axis=1)
print(time.time() - start, "secs")

79.57753705978394 secs


In [21]:
res

0      []
1      []
2      []
3      []
4      []
       ..
526    []
527    []
528    []
529    []
530    []
Length: 500, dtype: object

In [22]:
p, ptxt = extract_proj_descrip(proj_url(df, 0))
short_ptxt = shorten_list(ptxt)
clean_ptxt = clean_list(short_ptxt)
proj_desc = compress_list(clean_ptxt)
print(proj_desc)

TypeError: object of type 'NoneType' has no len()

In [ ]:
for i in range(5):
    print(i)
    p, ptxt = extract_proj_descrip(proj_url(df, i))
    short_ptxt = shorten_list(ptxt)
    clean_ptxt = clean_list(short_ptxt)
    compress_ptxt = compress_list(clean_ptxt)
    print(compress_ptxt)

In [ ]:
[w for w in ptxt if w != '\n']

In [ ]:
tst = [w for w in ptxt if w != '\n']

In [ ]:
tst

In [ ]:
shorten_list(ptxt)